In [1]:
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)

# New Data

In [2]:
df = pd.read_pickle('shoppersnew.pkl')

In [3]:
Info = df.sort_values('event_time').groupby(['product_id','user_id'])

In [4]:
df['event_time']= pd.to_datetime(df['event_time']) 

In [102]:
new_features= Info['event_type'].count().reset_index()

In [6]:
sales = df[df['event_time']<= '2020-01-31 23:59:59'].groupby('product_id')['event_type'].value_counts().unstack(fill_value = 0).loc[:,'purchase']


In [7]:
sales_per_month = (sales/4).reset_index()

In [8]:
sales_per_month = sales_per_month.rename(columns={'purchase':'Sales'})

In [103]:
new_features = pd.merge(new_features,sales_per_month, how='left',on = ['product_id'])

In [11]:
new_features

,product_id,user_id,event_type,Sales
0,3752,241524986,1,0.0
1,3752,246225692,2,0.0
2,3752,306131139,1,0.0
3,3752,382915461,1,0.0
4,3752,408427998,1,0.0
...,...,...,...,...
10157285,5932595,621638689,1,NaN
10157286,5932595,621827479,1,NaN
10157287,5932595,621829631,1,NaN
10157288,5932595,621885871,1,NaN


In [104]:
new_features['# Sessions'] = Info['user_session'].nunique().reset_index()['user_session']

In [11]:
Duration = Info['event_time'].agg(['max','min']).diff(axis= 1)

In [12]:
Duration_data = Duration['min'].dt.total_seconds().div(60).astype(int)

In [105]:
new_features['InteractionTime'] = abs(Duration_data.values)

In [14]:
purchase = Info['event_type'].value_counts().unstack(fill_value = 0).loc[:,'purchase']

In [106]:
new_features['last_time'] = Info['event_time'].max().values

In [107]:
new_features['max_price'] = Info['price'].max().values

In [108]:
new_features['min_price'] = Info['price'].min().values

In [109]:
new_features = new_features.fillna(0)

In [19]:
holiday = pd.read_csv('holidays.csv')

In [20]:
holiday['Date']= pd.to_datetime(holiday['Date']) 

In [22]:
holiday['Date'] = pd.DatetimeIndex(holiday['Date']).date

In [110]:
new_features['Date'] = pd.DatetimeIndex(new_features['last_time']).date

In [111]:
new_features = pd.merge(new_features,holiday, how='left',on = ['Date'])

In [112]:
new_features = new_features.fillna('None')

In [113]:
new_features['year'] = pd.DatetimeIndex(new_features['last_time']).year
new_features['month'] = pd.DatetimeIndex(new_features['last_time']).month
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
new_features['weekday'] = pd.DatetimeIndex(new_features['last_time']).weekday.map(dayOfWeek)

In [114]:
new_features['hour'] = pd.DatetimeIndex(new_features['last_time']).hour

In [115]:
new_features.rename(columns={'event_type':'# Clicks'})

,product_id,user_id,# Clicks,Sales,# Sessions,InteractionTime,last_time,max_price,min_price,Date,Holiday,year,month,weekday,hour
0,3752,241524986,1,0.0,1,0,2019-11-26 10:45:39,15.71,15.71,2019-11-26,None,2019,11,Tuesday,10
1,3752,246225692,2,0.0,2,56977,2020-02-28 19:33:09,15.71,15.71,2020-02-28,None,2020,2,Friday,19
2,3752,306131139,1,0.0,1,0,2019-10-01 12:11:11,15.71,15.71,2019-10-01,None,2019,10,Tuesday,12
3,3752,382915461,1,0.0,1,0,2019-11-26 06:09:46,15.71,15.71,2019-11-26,None,2019,11,Tuesday,6
4,3752,408427998,1,0.0,1,0,2019-12-06 20:52:35,15.71,15.71,2019-12-06,None,2019,12,Friday,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10157285,5932595,621638689,1,0.0,1,0,2020-02-28 21:42:41,0.00,0.00,2020-02-28,None,2020,2,Friday,21
10157286,5932595,621827479,1,0.0,1,0,2020-02-29 12:43:05,0.00,0.00,2020-02-29,None,2020,2,Saturday,12
10157287,5932595,621829631,1,0.0,1,0,2020-02-29 10:16:38,0.00,0.00,2020-02-29,None,2020,2,Saturday,10
10157288,5932595,621885871,1,0.0,1,0,2020-02-29 12:51:12,0.00,0.00,2020-02-29,None,2020,2,Saturday,12


# Category for hour

In [116]:
timeOfDay={0:'Dawn',1:'Dawn',2:'Dawn',3:'Dawn',4:'Dawn',5:'EarlyMorning',6:'EarlyMorning',7:'EarlyMorning',8:'EarlyMorning', 9:'Morning',10:'Morning',11:'Morning', 12:'Noon', 13:'Afternoon',14:'Afternoon',15:'Afternoon',16:'Afternoon', 17:'Evening',18:'Evening',19:'Evening',20:'Evening', 21:'Night',22:'Night',23:'Night', 24:'Midnight'}
new_features['timeOfDay'] = new_features.hour.map(timeOfDay)

# #Cart,View and Remove

In [30]:
cart = Info['event_type'].value_counts().unstack(fill_value = 0).loc[:,'cart']
view = Info['event_type'].value_counts().unstack(fill_value = 0).loc[:,'view']
remove = Info['event_type'].value_counts().unstack(fill_value = 0).loc[:,'remove_from_cart']

In [117]:
new_features['# Cart'] = cart.values
new_features['# View'] = view.values
new_features['# Remove'] = remove.values

# Total money of Cart, View, Remove

In [32]:
Num_in_cart = df.groupby('user_session')['event_type'].value_counts().unstack(fill_value = 0).loc[:,'cart']

In [33]:
Amount_view = df.groupby('user_session')['event_type'].value_counts().unstack(fill_value = 0).loc[:,'view']

In [34]:
Amount_remove = df.groupby('user_session')['event_type'].value_counts().unstack(fill_value = 0).loc[:,'remove_from_cart']

In [35]:
Amount_cart = Num_in_cart

In [48]:
Amount_cart.to_frame().reset_index()

,user_session,cart
0,0000061d-f3e9-484b-8c73-e54f355032a3,0
1,00000ac8-0015-4f12-996a-be2896323738,0
2,00000dd2-0f5d-4fc9-9d6b-2fc8c7514b04,0
3,000013d6-68a4-40cf-9452-6577dbfab515,1
4,00001aa1-7ee6-4d8c-81ff-bd5ce1dd1d6e,0
...,...,...
4535935,ffffeed4-b0dd-499c-ba8c-213a1ab9f50d,6
4535936,fffff319-41b2-4937-96bb-ac4aace7c300,0
4535937,fffff363-1257-4982-9e4f-da3b99b0719d,0
4535938,fffffa85-9968-4a7c-aac4-beb0a1992bbd,7


In [36]:
df = pd.merge(df,Amount_cart.to_frame().reset_index(), how='left',on = ['user_session'])

In [37]:
df = pd.merge(df,Amount_view.to_frame().reset_index(), how='left',on = ['user_session'])

In [38]:
df = pd.merge(df,Amount_remove.to_frame().reset_index(), how='left',on = ['user_session'])

In [39]:
insessioncart = df.groupby(['product_id','user_id'])['cart'].max()

In [40]:
insessionview = df.groupby(['product_id','user_id'])['view'].max()

In [41]:
insessionremove = df.groupby(['product_id','user_id'])['remove_from_cart'].max()

In [118]:
new_features = pd.merge(new_features,insessioncart, how='left',on = ['product_id','user_id'])

In [119]:
new_features = pd.merge(new_features,insessionview, how='left',on = ['product_id','user_id'])

In [120]:
new_features = pd.merge(new_features,insessionremove, how='left',on = ['product_id','user_id'])

In [121]:
new_features = new_features.rename(columns={'cart':'insession_cart','view':'insession_view','remove_from_cart':'insession_remove'})

In [122]:
new_features = new_features.drop(['product_id','user_id','last_time','Date'],axis = 1)

In [123]:
new_features['purchase']= purchase.values!=0

In [125]:
save = new_features.copy()

# one hot encoding

In [126]:
weekday = pd.get_dummies(new_features['weekday'])

In [127]:
weekday

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
10157285,1,0,0,0,0,0,0
10157286,0,0,1,0,0,0,0
10157287,0,0,1,0,0,0,0
10157288,0,0,1,0,0,0,0


In [128]:
new_features = pd.concat([new_features,weekday], axis=1)

In [131]:
timeofday = pd.get_dummies(new_features['timeOfDay'])

In [132]:
timeofday

,Afternoon,Dawn,EarlyMorning,Evening,Morning,Night,Noon
0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
10157285,0,0,0,0,0,1,0
10157286,0,0,0,0,0,0,1
10157287,0,0,0,0,1,0,0
10157288,0,0,0,0,0,0,1


In [133]:
new_features = pd.concat([new_features,timeofday], axis=1)

In [134]:
new_features.head()

,event_type,Sales,# Sessions,InteractionTime,max_price,min_price,Holiday,year,month,weekday,...,Thursday,Tuesday,Wednesday,Afternoon,Dawn,EarlyMorning,Evening,Morning,Night,Noon
0,1,0.0,1,0,15.71,15.71,None,2019,11,Tuesday,...,0,1,0,0,0,0,0,1,0,0
1,2,0.0,2,56977,15.71,15.71,None,2020,2,Friday,...,0,0,0,0,0,0,1,0,0,0
2,1,0.0,1,0,15.71,15.71,None,2019,10,Tuesday,...,0,1,0,0,0,0,0,0,0,1
3,1,0.0,1,0,15.71,15.71,None,2019,11,Tuesday,...,0,1,0,0,0,1,0,0,0,0
4,1,0.0,1,0,15.71,15.71,None,2019,12,Friday,...,0,0,0,0,0,0,1,0,0,0


In [135]:
new_features.drop('weekday',axis=1,inplace=True)

In [ ]:
new_features.to_csv('features.csv')